### FAP tests

We are testing against the null hypothesis that the white dwarf found was a chance alignment of a random source that just happened to meet our criteria (i.e. appears co-moving with the pulsar, has properties consistent with being a WD) with the pulsar position on the sky.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astropy.table import Table, unique
from tqdm import tqdm
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [3]:
pulsar_data = np.genfromtxt('ipta_3arcsec_wpm-result.csv', 
                            delimiter=',', names=True, dtype=None, encoding=None)

In [4]:
pulsar_data = Table(pulsar_data)
pulsar_data.sort(['names', 'angdist'])
#pulsar_data = unique(pulsar_data, keys='names') # drop less-good duplicate matches

This function will reproduce the cuts that we made to determine which nearby sources were candidate matches and which should be thrown out:

**NOTE: absolute magnitude > 7 cut was made relatively arbitrarily to weed out main-sequence stars from white dwarfs - this may need to be changed depending on what criteria we settle on for the paper.**

In [5]:
def make_cuts(tbl, pulsar_pmra, pulsar_pmdec):
    # take an astropy table, apply cuts
    pmra_tol = 3. * tbl['pmra_error'] # 3 sigma
    pmdec_tol = 3. * tbl['pmdec_error'] # 3 sigma
    idx = np.isfinite(tbl['parallax']) & (np.abs(tbl['pmra'] - pulsar_pmra) <= pmra_tol) \
            & (np.abs(tbl['pmdec'] - pulsar_pmdec) <= pmdec_tol) 
#            & ((tbl['phot_g_mean_mag'] + 5.*np.log10(tbl['parallax']) - 10.) >= 5.)
    return tbl[idx]

In [7]:
make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1'])['names']

/anaconda2/envs/py3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, op)(other)


J0437-4715
J1012+5307
J1024-0719
J1732-5049
J1843-1113
J1910+1256


Now we'll randomize each pulsar's coordinates (within 3 degrees in RA and Dec to reflect the actual crowding in the general sky region) and see how many objects in the new area pass our cuts.

In [ ]:
pulsars_to_test = make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1']) # just the good matches
faps = []
N_trials = 10000
search_radius = u.Quantity(3.0, u.arcsec)
for p in pulsars_to_test:
    #print('calculating FAP for pulsar {0}...'.format(p['names']))
    n_found = np.zeros(N_trials)
    for n in tqdm(range(N_trials)):
        test_ra = p['ra_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
        test_dec = p['dec_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
        test_coord = SkyCoord(ra=test_ra, dec=test_dec, unit=(u.degree, u.degree))
        j = Gaia.cone_search_async(test_coord, search_radius, verbose=False)
        r = j.get_results()
        n_found[n] = len(make_cuts(r, p['pmra_1'], p['pmdec_1']))
    #print('found FAP of {0} in {1}.'.format(np.sum(n_found > 0), N_trials))
    faps.append(np.sum(n_found > 0)/N_trials)

/anaconda2/envs/py3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, op)(other)
  0%|          | 1/10000 [00:01<2:52:36,  1.04s/it]

Query finished.


  0%|          | 2/10000 [00:01<2:26:27,  1.14it/s]

Query finished.


  0%|          | 3/10000 [00:02<2:19:18,  1.20it/s]

...

Query finished.


 32%|███▏      | 3163/10000 [1:14:13<2:40:26,  1.41s/it]

Query finished.


 32%|███▏      | 3164/10000 [1:14:15<2:40:26,  1.41s/it]

Query finished.


In [ ]:
for p,f in zip(pulsars_to_test, faps):
    print("{0}: FAP {1}".format(p['names'], f))